In [1]:
import numpy as np
import os
import requests
import pyxis as px

# Let us start downloading the data

We will use the MNIST dataset, this version is already in numpy array and is provided by tensorflow.

In [2]:
# Downloads dataset
if not os.path.exists("mnist.npz"):
    with open("mnist.npz", 'wb') as fout:
        response = requests.get("https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz", stream=True)
        response.raise_for_status()
        for block in response.iter_content(4096):
            fout.write(block)
# Loads arrays

with np.load("./mnist.npz") as f:
    x_train, y_train = f['x_train'], f['y_train']


## Creates train database

We will begin by creating a small dataset to test with. It will consist of `60000` samples from the training partion of mnist.

In [3]:
with px.Writer(dirpath='mnist_train', map_size_limit=256, ram_gb_limit=1) as db:
    db.put_samples('X', x_train.astype(np.float32), 'y', y_train.astype(np.long))

### Checks dataset

In [4]:
with px.Reader('mnist_train') as db:
    print(db)


pyxis.Reader
Location:		'mnist_train'
Number of samples:	60000
Data keys (0th sample):
	'X' <- dtype: float32, shape: (28, 28)
	'y' <- dtype: int64, shape: ()


# Train 1 epoch using pytorch dataloader

We will train 1 epoch using the normal dataloader and torch.utils.data.DataLoader

In [5]:
try:
    import torch
    import torch.utils.data
except ImportError:
    raise ImportError('Could not import the PyTorch library `torch` or '
                      '`torch.utils.data`. Please refer to '
                      'https://pytorch.org/ for installation instructions.')
import pyxis.torch as pxt

#### Initialize pytorch methods

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Not optimal network, but big enough to simulate some gradient load
network = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(28*28,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,10)) 
network.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(network.parameters(), lr=1e-5, weight_decay=1e-2)


### Initialize classical dataloaders

In [7]:
dataset = pxt.TorchDataset('mnist_train')
use_cuda = True and torch.cuda.is_available()
kwargs = {"num_workers": 2, "pin_memory": True} if use_cuda else {}
loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False,**kwargs)


In [8]:
def train_epoch():
    for i, d in enumerate(loader):
        data,target = d['X'],d['y']
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print('Train[{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            i * len(data), len(loader.dataset),
            100. * i / len(loader), loss.item()))

In [9]:
%timeit train_epoch()

Train[59968/60000 (100%)]	Loss: 0.682663
Train[59968/60000 (100%)]	Loss: 0.292598
Train[59968/60000 (100%)]	Loss: 0.182662
Train[59968/60000 (100%)]	Loss: 0.138513
Train[59968/60000 (100%)]	Loss: 0.125213
Train[59968/60000 (100%)]	Loss: 0.114652
Train[59968/60000 (100%)]	Loss: 0.108128
Train[59968/60000 (100%)]	Loss: 0.103621
4.44 s ± 62.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Initialize pytorch methods again 

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Not optimal network, but big enough to simulate some gradient load
network = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(28*28,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,10)) 
network.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(network.parameters(), lr=1e-5, weight_decay=1e-2)


### Initialize pyxis iterator

In [11]:
iterator = pxt.TorchIterator(
    device = device,
    dir_path = 'mnist_train',
    keys=('X','y'),
    batch_size=32,
    num_worker=2,
    pre_fetcher_queue=100,
    device_transfer_queue=2,
)

In [12]:
def train_epoch_with_pyxis():
    for i, d in enumerate(iterator):
        data,target = d
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print('Train[{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            i * len(data), len(loader.dataset),
            100. * i / len(loader), loss.item()))

In [13]:
%timeit train_epoch_with_pyxis()

Train[599680/60000 (100%)]	Loss: 0.418909
Train[599680/60000 (100%)]	Loss: 0.345567
Train[599680/60000 (100%)]	Loss: 0.131335
Train[599680/60000 (100%)]	Loss: 0.163470
Train[599680/60000 (100%)]	Loss: 0.200821
Train[599680/60000 (100%)]	Loss: 0.117903
Train[599680/60000 (100%)]	Loss: 0.149772
Train[599680/60000 (100%)]	Loss: 0.104580
3.6 s ± 67.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
